In [43]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

This code example implements back-propagation algorithm for a two-level network and demonstrates how to use it to learn the exclusive OR (XOR) function. The network has two neurons in the hidden layer and a single output neuron. More context for this code example can be found in the section "Programming Example: Learning the XOR Function" in Chapter 3 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).


The initialization code in the code snippet below is similar to what we did for the perceptron example c1e1_perceptron_learning. One thing to note is that we have started to use NumPy arrays so that we can make use of some NumPy functionality. The same holds for our random number generator (we call np.random.seed instead of just random.seed).

For the training examples, we have now changed the ground truth to be between 0.0 and 1.0 because we have decided to use the logistic sigmoid function as an activation function for the output neuron, and its output range does not go to −1.0 as the perceptron did.


In [44]:
import numpy as np

np.random.seed(3) # To make repeatable
LEARNING_RATE = 0.1
index_list = [0, 1, 2, 3] # Used to randomize order

# Define training examples.
x_train = [np.array([1.0, -1.0, -1.0]),
           np.array([1.0, -1.0, 1.0]),
           np.array([1.0, 1.0, -1.0]),
           np.array([1.0, 1.0, 1.0])]
y_train = [0.0, 1.0, 1.0, 0.0] # Output (ground truth)


In the next code snippet, we declare variables to hold the state of our three neurons. A real implementation would typically be parameterized to be able to choose number of inputs, layers, and number of neurons in each layer, but all of those parameters are hardcoded in this example to focus on readability.


In [45]:
def neuron_w(input_count):
    weights = np.zeros(input_count+1)
    for i in range(1, (input_count+1)):
        weights[i] = np.random.uniform(-1.0, 1.0)
    return weights

n_w = [neuron_w(2), neuron_w(2), neuron_w(2), neuron_w(3)]
n_y = [0, 0, 0, 0]
n_error = [0, 0, 0, 0]


The next code snippet starts with a function to print all the nine weights of the network (each print statement prints a three-element weight vector). The forward_pass function first computes the outputs of neurons 0 and 1 with the same inputs (the inputs from the training example) and then puts their outputs into an array, together with a bias value of 1.0, to use as input to neuron 2. That is, this function defines the topology of the network. We use tanh for the neurons in the first layer and the logistic sigmoid function for the output neuron.

The backward_pass function starts by computing the derivative of the error function and then computes the derivative of the activation function for the output neuron. The error term of the output neuron is computed by multiplying these two together. We then continue to backpropagate the error to each of the two neurons in the hidden layer. This is done by computing the derivatives of their activation functions and multiplying these derivatives by the error term from the output neuron and by the weight to the output neuron.

Finally, the adjust_weights function adjusts the weights for each of the three neurons. The adjustment factor is computed by multiplying the input by the learning rate and the error term for the neuron in question.


In [46]:
def show_learning():
    global epoch
    print(f'Epoch: {epoch}')
    epoch = epoch + 1
    print('Current weights:')
    for i, w in enumerate(n_w):
        for j, wj in enumerate(w):
            print('neuron ', i, ' weight ', j, ':', '%5.2f' % wj)
        print('----------------')

def forward_pass(x):
    global n_y
    n_y[0] = np.tanh(np.dot(n_w[0], x)) # Neuron 0
    n_y[1] = np.tanh(np.dot(n_w[1], x)) # Neuron 1
    n_y[2] = np.tanh(np.dot(n_w[2], x))
    n2_inputs = np.array([1.0, n_y[0], n_y[1], n_y[2]]) # 1.0 is bias
    z2 = np.dot(n_w[3], n2_inputs)
    n_y[3] = 1.0 / (1.0 + np.exp(-z2)) # Apply sigmoid activation in the output layer

def backward_pass(y_truth):
    global n_error
    error_prime = -(y_truth - n_y[3]) # Derivative of loss-func
    derivative = n_y[3] * (1.0 - n_y[3]) # Logistic derivative
    n_error[3] = error_prime * derivative
    
    derivative = 1.0 - n_y[0]**2 # tanh derivative
    n_error[0] = n_w[3][1] * n_error[3] * derivative
    
    derivative = 1.0 - n_y[1]**2 # tanh derivative
    n_error[1] = n_w[3][2] * n_error[3] * derivative
    
    derivative = 1.0 - n_y[2]**2 # tanh derivative
    n_error[2] = n_w[3][3] * n_error[3] * derivative

def adjust_weights(x):
    global n_w
    n_w[0] -= (x * LEARNING_RATE * n_error[0])
    n_w[1] -= (x * LEARNING_RATE * n_error[1])
    n_w[2] -= (x * LEARNING_RATE * n_error[2])
    n2_inputs = np.array([1.0, n_y[0], n_y[1], n_y[2]]) # 1.0 is bias
    n_w[3] -= (n2_inputs * LEARNING_RATE * n_error[3])


With all these pieces in place, the only remaining piece is the training loop shown in the code snippet below, which is somewhat similar to the training loop for the perceptron example in c1e1_perceptron_learning.

We pick training examples in random order, call the functions forward_pass, backward_pass, and adjust_weights, and then print out the weights with the function show_learning. We adjust the weights regardless whether the network predicts correctly or not. Once we have looped through all four training examples, we check whether the network can predict them all correctly, and if not, we do another pass over them in random order.


In [47]:
# Network training loop.
all_correct = False
epoch = 0
while not all_correct: # Train until converged
    all_correct = True
    np.random.shuffle(index_list) # Randomize order
    for i in index_list: # Train on all examples
        forward_pass(x_train[i])
        backward_pass(y_train[i])
        adjust_weights(x_train[i])
        show_learning() # Show updated weights
    for i in range(len(x_train)): # Check if converged
        forward_pass(x_train[i])
        print('x1 =', '%4.1f' % x_train[i][1], ', x2 =',
              '%4.1f' % x_train[i][2], ', y =',
              '%.4f' % n_y[2])
        if(((y_train[i] < 0.5) and (n_y[3] >= 0.5))
                or ((y_train[i] >= 0.5) and (n_y[3] < 0.5))):
            all_correct = False


Epoch: 0
Current weights:
neuron  0  weight  0 : -0.05
neuron  0  weight  1 :  0.15
neuron  0  weight  2 :  0.37
----------------
neuron  1  weight  0 : -0.04
neuron  1  weight  1 : -0.38
neuron  1  weight  2 : -0.01
----------------
neuron  2  weight  0 : -0.07
neuron  2  weight  1 :  0.85
neuron  2  weight  2 :  0.73
----------------
neuron  3  weight  0 :  0.07
neuron  3  weight  1 : -0.73
neuron  3  weight  2 : -0.56
neuron  3  weight  3 : -0.90
----------------
Epoch: 1
Current weights:
neuron  0  weight  0 : -0.08
neuron  0  weight  1 :  0.12
neuron  0  weight  2 :  0.40
----------------
neuron  1  weight  0 : -0.06
neuron  1  weight  1 : -0.41
neuron  1  weight  2 :  0.01
----------------
neuron  2  weight  0 : -0.11
neuron  2  weight  1 :  0.81
neuron  2  weight  2 :  0.77
----------------
neuron  3  weight  0 :  0.12
neuron  3  weight  1 : -0.74
neuron  3  weight  2 : -0.57
neuron  3  weight  3 : -0.89
----------------
Epoch: 2
Current weights:
neuron  0  weight  0 : -0.06
neu

# Potential limitations about the architecture
- As shown in the input, the model takes some time to converge. This can be due to the fact that a small learning rate is changing the weights in a slow manner.
- Changing the learning rate to a bigger value could help the model to converge faster, but it also can overshoot the ideal weight values, affecting accuracy.
- As the weights are initialized completely randomly, this can also be a problem in the model's convergence.
    - For example, if the initialized weights are too small, each multiplication reduces the gradient even further, and that can lead to the vanishing gradient problem.

# Activation functions
- In this architecture, tanh was used in the input layer and sigmoid was used in the output layer.
- These activation functions are important to help the network to learn complex, non-linear relationships. Without activation functions, the model would act like a linear regression, regardless of the number of neurons or trainable parameters.
- Activation functions are also important to normalize the range of the outputs of neurons, making the outputs manageable and interpretable.
- For example, the differences between the tanh function and the sigmoid are:
    - The range is an activation function with a range centered around 0, that being between -1 and 1, while sigmoid returns a range between 0 and 1.
- Tanh is preferred over sigmoid in the hidden layers because it is centered around 0, and this provides strong gradients for mid-range inputs. However, both tanh and sigmoid have the risk of outputting extremely low gradients, and thus leading the model to the vanishing gradient problem.